In [ ]:
import pandas as pd

#import
df = pd.read_csv("dataset.csv")

Tokenization

In [ ]:
import nltk

df['combined_text'] = df['combined_text'].astype(str)
df['tokenized_text'] = df['combined_text'].apply(nltk.word_tokenize)


word2vec

In [ ]:
import numpy as np
from gensim.models import Word2Vec

model = Word2Vec(sentences=df['tokenized_text'], vector_size=1000, window=5, min_count=1, workers=4) #train on all texts 

def get_embedding(tokens):
    valid_tokens = [token for token in tokens if token in model.wv.index_to_key]
    
    if valid_tokens:
        return np.mean([model.wv[token] for token in valid_tokens], axis=0)
    else:
        return np.zeros(model.vector_size)

df['text_embedding'] = df['tokenized_text'].apply(get_embedding)
print(df[['combined_text', 'text_embedding']])


In [ ]:
df_label_0 = df[df['label'] == 0]
df_label_1 = df[df['label'] == 1]
df_label_2 = df[df['label'] == 2]

# Total dataset size and test set calculation
total_size = len(df)
test_set_size = int(total_size * 0.20)  # 20% of the total dataset for the test set
test_samples_label_1_size = int(test_set_size * 0.15)  # 15% of the test set for label 1
test_samples_label_2_size = int(test_set_size * 0.05)  # 5% of the test set for label 2

test_samples_label_1 = df_label_1.sample(n=test_samples_label_1_size, random_state=42)
test_samples_label_2 = df_label_2.sample(n=test_samples_label_2_size, random_state=42)

remaining_test_size = test_set_size - (test_samples_label_1_size + test_samples_label_2_size)

additional_test_samples = df_label_0.sample(n=remaining_test_size, random_state=42)

test_df = pd.concat([test_samples_label_1, test_samples_label_2, additional_test_samples])

df_remaining = df.drop(test_df.index)

X_train_embeddings = np.stack(df_remaining['text_embedding'].values)
y_train = df_remaining['label'].values

# test set
X_test_embeddings = np.stack(test_df['text_embedding'].values)
y_test = test_df['label'].values

# training dataset
X_train, y_train = df_remaining.drop('label', axis=1), df_remaining['label']

# Verification of sizes
(len(test_df), len(X_train_embeddings), test_samples_label_1_size, test_samples_label_2_size)



In [ ]:
# Save the training set
import pickle
with open('X_train.pkl', 'wb') as file:
    pickle.dump(X_train_embeddings, file)

with open('y_train.pkl', 'wb') as file:
    pickle.dump(y_train, file)

with open('X_eval.pkl', 'wb') as file:
    pickle.dump(X_test_embeddings, file)

with open('y_eval.pkl', 'wb') as file:
    pickle.dump(y_test, file)

Create pickles for transformers with text not embeddings


In [ ]:
import numpy as np

df_label_0 = df_text[df_text['label'] == 0]
df_label_1 = df_text[df_text['label'] == 1]
df_label_2 = df_text[df_text['label'] == 2]

# Total dataset size and test set calculation
total_size = len(df_text)
test_set_size = int(total_size * 0.20)  # 20% of the total dataset for the test set
test_samples_label_1_size = int(test_set_size * 0.15)  # 15% of the test set for label 1
test_samples_label_2_size = int(test_set_size * 0.05)  # 5% of the test set for label 2

test_samples_label_1 = df_label_1.sample(n=test_samples_label_1_size, random_state=42)
test_samples_label_2 = df_label_2.sample(n=test_samples_label_2_size, random_state=42)

remaining_test_size = test_set_size - (test_samples_label_1_size + test_samples_label_2_size)

additional_test_samples = df_label_0.sample(n=remaining_test_size, random_state=42)

test_df = pd.concat([test_samples_label_1, test_samples_label_2, additional_test_samples])

df_remaining = df.drop(test_df.index)

X_train_text = np.stack(df_remaining['combined_text'].values)
y_train = df_remaining['label'].values

# test set
X_test_text = np.stack(test_df['combined_text'].values)
y_test = test_df['label'].values

# training dataset
X_train, y_train = df_remaining.drop('label', axis=1), df_remaining['label']

# Verification of sizes
(len(test_df), len(X_train_text), test_samples_label_1_size, test_samples_label_2_size)

In [ ]:
import pickle
with open('X_train_text.pkl', 'wb') as file:
    pickle.dump(X_train_text, file)

with open('y_train_text.pkl', 'wb') as file:
    pickle.dump(y_train, file)

with open('X_eval_text.pkl', 'wb') as file:
    pickle.dump(X_test_text, file)

with open('y_eval_text.pkl', 'wb') as file:
    pickle.dump(y_test, file)